In [2]:
import catboost

In [3]:
import pandas as pd

In [4]:
train_set = pd.read_csv('train.csv', sep='\t')

In [5]:
train_set[:5]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,B2C,25454,3.0,2019-03-26 15:11:00.000-07:00,0,0.0,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,5,1,LETTER,1
1,C2C,6727381,2.0,2018-06-02 12:53:00.000-07:00,0,3.0,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,0,1,PACKAGE_THICK_ENVELOPE,2
2,B2C,18507,1.0,2019-01-07 16:22:00.000-05:00,0,4.5,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,9,1,PACKAGE_THICK_ENVELOPE,3
3,B2C,4677,1.0,2018-12-17 16:56:00.000-08:00,0,0.0,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,8,1,PACKAGE_THICK_ENVELOPE,4
4,B2C,4677,1.0,2018-07-27 16:48:00.000-07:00,0,0.0,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,3,1,PACKAGE_THICK_ENVELOPE,5


In [6]:
train_set.shape

(15000000, 19)

In [7]:
train_set['weight_units'].sum()

15000036

In [8]:
train_set[train_set['weight_units'] != 1]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
115821,B2C,46815,1.0,2018-01-29 10:06:00.000-06:00,0,0.00,3,5,35503,30605,1,6.89,2,2018-01-28 20:18:59.000-06:00,2018-01-31,113,2,LARGE_ENVELOPE,115822
847218,B2C,40624,1.0,2019-10-03 13:18:00.000-05:00,2,0.00,2,9,63044,85715,6,26.99,1,2019-10-03 01:28:56.000-05:00,2019-10-10,0,2,NONE,847219
908707,C2C,295500,1.0,2018-03-21 14:00:00.000-04:00,5,7.00,2,5,02050,02109,1,29.30,1,2018-03-04 08:51:33.000-05:00,2018-03-07,13,2,LARGE_ENVELOPE,908708
1119090,B2C,98438,1.0,2018-07-25 11:38:00.000-05:00,0,0.00,3,5,65024,59901,1,38.50,1,2018-07-24 15:08:22.000-05:00,2018-07-30,0,2,LETTER,1119091
1119891,B2C,98438,1.0,2019-02-01 11:50:00.000-06:00,0,0.00,3,5,65024,75077,1,26.50,1,2019-01-31 16:12:32.000-06:00,2019-02-04,0,2,LETTER,1119892
1136383,B2C,40624,1.0,2019-07-02 13:15:00.000-05:00,2,0.00,2,9,63044,94610,6,37.99,2,2019-07-01 19:38:51.000-05:00,2019-07-06,0,2,NONE,1136384
1518882,B2C,24868,1.0,2019-10-02 11:56:00.000-04:00,0,0.00,3,5,14609,92107,1,5.51,1,2019-10-01 13:49:48.000-04:00,2019-10-15,0,2,LETTER,1518883
2229000,B2C,98438,1.0,2018-08-07 11:50:00.000-05:00,0,0.00,3,5,65024,62278,1,14.00,1,2018-08-06 15:10:08.000-05:00,2018-08-08,0,2,LETTER,2229001
2330268,B2C,14567,1.0,2019-03-07 22:17:00.000-08:00,0,0.00,3,5,91748,77045,1,10.85,1,2019-03-06 12:50:00.000-08:00,2019-03-11,0,2,PACKAGE_THICK_ENVELOPE,2330269
2561431,B2C,40953,1.0,2019-05-16 21:13:00.000-07:00,1,0.00,2,5,06010-6670,20716,1,746.10,1,2019-05-16 10:07:31.000-07:00,2019-05-18,0,2,PACKAGE_THICK_ENVELOPE,2561432


In [10]:
quiz_set = pd.read_csv('quiz.tsv', sep='\t')

In [11]:
quiz_set[:5]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,B2C,260031,1.0,2019-11-15 05:11:00.000-07:00,0,0.0,3,5,84116,34788,2,28.00,1,2019-11-13 19:28:44.000-07:00,NaN,4,1,PACKAGE_THICK_ENVELOPE,15000001
1,B2C,4854,1.0,2019-11-28 02:03:00.000-05:00,0,0.0,3,5,04901,54904,9,9.95,1,2019-11-26 14:36:21.000-05:00,NaN,13,1,PACKAGE_THICK_ENVELOPE,15000002
2,B2C,1987,1.0,2019-12-03 00:02:00.000-08:00,0,0.0,3,5,90015,80918,2,16.25,1,2019-12-02 07:39:41.000-08:00,NaN,6,1,PACKAGE_THICK_ENVELOPE,15000003
3,B2C,43799,1.0,2019-12-14 19:56:00.000-05:00,0,0.0,3,5,04007,49036,16,13.65,1,2019-12-12 14:57:31.000-05:00,NaN,4,1,PACKAGE_THICK_ENVELOPE,15000004
4,B2C,3660,0.0,2019-12-19 14:30:00.000-08:00,0,0.0,3,5,95841,74074,1,13.99,1,2019-12-18 16:26:43.000-08:00,NaN,2,1,LARGE_ENVELOPE,15000005


In [14]:
zip_code = set(train_set['item_zip']) | set(train_set['buyer_zip']) | set(quiz_set['item_zip']) | set(quiz_set['buyer_zip'])

In [15]:
len(zip_code)

78535

In [16]:
import json

In [17]:
json.dump(list(zip_code), open('zip_code_list.json', 'w', encoding='utf-8'))

In [22]:
train_set[train_set['item_zip'].isnull()]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
5914552,C2C,5971014,NaN,2018-12-15 15:43:05.000-08:00,2,0.0,2,9,NaN,80219-3401,8,302.3,1,2018-12-09 16:53:56.000-08:00,2018-12-18,0,1,PACKAGE_THICK_ENVELOPE,5914553


In [23]:
train_set[train_set['buyer_zip'].isnull()]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
4845032,C2C,1343135,NaN,2018-07-30 11:34:00.000-05:00,12,0.0,-1,-1,65804,NaN,4,5.5,1,2018-07-28 17:03:33.000-05:00,2018-08-04,5,1,PACKAGE_THICK_ENVELOPE,4845033


In [20]:
quiz_set['item_zip'].isnull().sum()

0

In [21]:
quiz_set['item_zip'].isnull().sum()

0